In [1]:
using ITensors, TensorKit, TensorOperations

In [58]:
let
    L = 6
    sites = siteinds("Qudit",L; dim=6, conserve_qns=true, conserve_number=true)
    # sites = siteinds("Qudit",L; dim=5)
    t = 0.1
    U = 2
    μ = 0

    H = OpSum()
    for i in (1:L-1)
        H += -μ - U/2, "n", i
        H += U/2, "n", i, "n", i
        H += -t, "a", i, "adag", i+1
        H += -t, "adag", i, "a", i+1
    end
    # periodic boundary conditions.
    H += -t, "a", 1, "adag", L
    H += -t, "adag", 1, "a", L
    print(H)
    Ham = MPO(H, sites)

    state = [isodd(n) ? "1" : "1" for n=1:L]
    psi0 = randomMPS(sites,state)
  
    nsweeps = 10
    maxdim = [40, 80, 400, 400, 800, 800, 1000, 1000, 1000, 1000]
    cutoff = [1e-12]
  
    energy, psi = dmrg(Ham,psi0; nsweeps, maxdim, cutoff)
    N = expect(psi, "n")
    println(N)
    println(sum(N))
    return
end

sum(
  -1.0 n(1,)
  1.0 n(1,) n(1,)
  -0.1 a(1,) adag(2,)
  -0.1 adag(1,) a(2,)
  -1.0 n(2,)
  1.0 n(2,) n(2,)
  -0.1 a(2,) adag(3,)
  -0.1 adag(2,) a(3,)
  -1.0 n(3,)
  1.0 n(3,) n(3,)
  -0.1 a(3,) adag(4,)
  -0.1 adag(3,) a(4,)
  -1.0 n(4,)
  1.0 n(4,) n(4,)
  -0.1 a(4,) adag(5,)
  -0.1 adag(4,) a(5,)
  -1.0 n(5,)
  1.0 n(5,) n(5,)
  -0.1 a(5,) adag(6,)
  -0.1 adag(5,) a(6,)
  -0.1 a(1,) adag(6,)
  -0.1 adag(1,) a(6,)
)After sweep 1 energy=-0.41506019333397914  maxlinkdim=18 maxerr=3.45E-18 time=0.083


After sweep 2 energy=-0.6082942519569269  maxlinkdim=29 maxerr=3.68E-13 time=0.149


After sweep 3 energy=-0.6729488323008808  maxlinkdim=30 maxerr=0.00E+00 time=0.155


After sweep 4 energy=-0.6875208071953002  maxlinkdim=29 maxerr=1.22E-13 time=0.157


After sweep 5 energy=-0.6916032619605623  maxlinkdim=29 maxerr=8.00E-14 time=0.150


...

After sweep 7 energy=-0.6926858821375497  maxlinkdim=27 maxerr=3.87E-13 time=0.157


After sweep 8 energy=-0.6927036856074449  maxlinkdim=27 maxerr=6.62E-13 time=0.144


After sweep 9 energy=-0.6927066286197037  maxlinkdim=25 maxerr=8.61E-13 time=0.139


After sweep 10 energy=-0.6927072397659445  maxlinkdim=25 maxerr=8.24E-13 time=0.166


[0.5355692214120122, 0.45582763457878683, 0.4806788206004547, 0.4557957630945251, 0.5355898475622978, 3.5365387127519257]
6.000000000000002
